In [136]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import datetime

from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_portfolio import StockPortfolioEnv
import plotly
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

from finrl.model.models import DRLAgent
from finrl.trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts
import matplotlib.pyplot as plt 
%matplotlib inline
import sys
sys.path.append("../FinRL-Library")

# Get data

In [137]:
def get_price(assets):
    # config.DOW_30_TICKER
    df = YahooDownloader(start_date = '1995-01-01',
                         end_date = '2030-01-01',
                         ticker_list =assets ).fetch_data() #,'LQD','IEO','GLD'

    price = df.set_index(['date','tic'])[['close']].unstack()
    price.columns = price.columns.droplevel()
    price.index = pd.to_datetime(price.index)
    daily_rts = price/price.shift(1) -1
    return price, daily_rts

In [138]:
# # resample into weekly
# price = price.resample('W').first()
# weekly_rts = price/price.shift(1)-1

# Backtest 

In [139]:

def backtest(price,weights, rf): 
    """both price and weights needs to be in daily"""
    asset_class = list(weights.columns.unique())
    returns = (price/price.shift(1)-1).resample('B').last()
    returns = returns.loc[weights.index[0]:weights.index[-1]]
    weights = weights.resample('B').last()

    returns['portfolio_returns'] = (returns*weights).sum(axis = 1)
    returns['portfolio_cum_returns'] = (returns['portfolio_returns']+1).cumprod() -1
    port_mean_returns = returns['portfolio_returns'].mean()*260
    port_total_returns =  returns['portfolio_cum_returns'][-1]
    port_sigma = returns['portfolio_returns'].std() * np.sqrt(260)
    port_sharpe = (port_mean_returns - rf)/ port_sigma
    print('Asset class:')
    print(asset_class)
    print('Mean returns: {}, total returns: {}, sharpe: {}'.format(port_mean_returns,port_total_returns,port_sharpe))
    returns['portfolio_cum_returns'].iplot(title = 'cumulative returns');
    return returns

# Models

## Hierarchical risk parity

In [140]:
import pandas as pd
import numpy as np
import warnings
import six
import matplotlib.pyplot as plt
import scipy.optimize as sco 
from copy import deepcopy
import itertools
from datetime import timedelta
from pandas.tseries.offsets import BDay
warnings.filterwarnings('ignore')


import scipy.cluster.hierarchy as sch
import numpy as np
import pandas as pd
from datetime import date
from matplotlib import pyplot as plt
import cvxopt as opt
from cvxopt import blas, solvers

In [141]:

def getIVP(cov, **kargs):
    # Compute the inverse-variance portfolio
    ivp = 1. / np.diag(cov)
    ivp /= ivp.sum()
    return ivp


def getClusterVar(cov,cItems):
    # Compute variance per cluster
    cov_=cov.loc[cItems,cItems] # matrix slice
    w_=getIVP(cov_).reshape(-1,1)
    cVar=np.dot(np.dot(w_.T,cov_),w_)[0,0]
    return cVar


def getQuasiDiag(link):
    # Sort clustered items by distance
    link = link.astype(int)
    sortIx = pd.Series([link[-1, 0], link[-1, 1]])
    numItems = link[-1, 3]  # number of original items
    while sortIx.max() >= numItems:
        sortIx.index = range(0, sortIx.shape[0] * 2, 2)  # make space
        df0 = sortIx[sortIx >= numItems]  # find clusters
        i = df0.index
        j = df0.values - numItems
        sortIx[i] = link[j, 0]  # item 1
        df0 = pd.Series(link[j, 1], index=i + 1)
        sortIx = sortIx.append(df0)  # item 2
        sortIx = sortIx.sort_index()  # re-sort
        sortIx.index = range(sortIx.shape[0])  # re-index
    return sortIx.tolist()


def getRecBipart(cov, sortIx):
    # Compute HRP alloc
    w = pd.Series(1, index=sortIx)
    cItems = [sortIx]  # initialize all items in one cluster
    while len(cItems) > 0:
        cItems = [i[j:k] for i in cItems for j, k in ((0, len(i) // 2), (len(i) // 2, len(i))) if len(i) > 1]  # bi-section
        for i in range(0, len(cItems), 2):  # parse in pairs
            cItems0 = cItems[i]  # cluster 1
            cItems1 = cItems[i + 1]  # cluster 2
            cVar0 = getClusterVar(cov, cItems0)
            cVar1 = getClusterVar(cov, cItems1)
            alpha = 1 - cVar0 / (cVar0 + cVar1)
            w[cItems0] *= alpha  # weight 1
            w[cItems1] *= 1 - alpha  # weight 2
    return w


def correlDist(corr):
    # A distance matrix based on correlation, where 0<=d[i,j]<=1
    # This is a proper distance metric
    dist = ((1 - corr) / 2.)**.5  # distance matrix
    return dist


def getHRP(cov, corr):
    # Construct a hierarchical portfolio
    dist = correlDist(corr)
    link = sch.linkage(dist, 'single')
    #dn = sch.dendrogram(link, labels=cov.index.values, label_rotation=90)
    #plt.show()
    sortIx = getQuasiDiag(link)
    sortIx = corr.index[sortIx].tolist()
    hrp = getRecBipart(cov, sortIx)
    return hrp.sort_index()

def getMVP(cov):

    cov = cov.T.values
    n = len(cov)
    N = 100
    mus = [10 ** (5.0 * t / N - 1.0) for t in range(N)]

    # Convert to cvxopt matrices
    S = opt.matrix(cov)
    #pbar = opt.matrix(np.mean(returns, axis=1))
    pbar = opt.matrix(np.ones(cov.shape[0]))

    # Create constraint matrices
    G = -opt.matrix(np.eye(n))  # negative n x n identity matrix
    h = opt.matrix(0.0, (n, 1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)

    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu * S, -pbar, G, h, A, b)['x']
                  for mu in mus]
    ## CALCULATE RISKS AND RETURNS FOR FRONTIER
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S * x)) for x in portfolios]
    ## CALCULATE THE 2ND DEGREE POLYNOMIAL OF THE FRONTIER CURVE
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    # CALCULATE THE OPTIMAL PORTFOLIO
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']

    return list(wt)

def get_all_portfolios(returns):
    
    cov, corr = returns.cov(), returns.corr()
    hrp = getHRP(cov, corr)
    ivp = getIVP(cov)
    ivp = pd.Series(ivp, index=cov.index)
    mvp = getMVP(cov)
    mvp = pd.Series(mvp, index=cov.index)
    
    portfolios = pd.DataFrame([mvp, ivp, hrp], index=['MVP', 'IVP', 'HRP']).T
    
    return portfolios

In [142]:
def hrp_weight_generator(training_window,allocation_window,df):

    all_weights = pd.DataFrame()
    #loop the dates based on allocation window
    for i in range( training_window, len(df), allocation_window):
        training_df = df.iloc[i - training_window: i ].fillna(0.0001*2)
        hrp = get_all_portfolios(training_df)
        hrpdic = dict(zip(hrp.index, hrp['HRP']))

        #generate a weights dataframe for each allocation window
        weights_df = pd.DataFrame(data = hrpdic ,
                     columns = training_df.columns, 
                                # index = df.index[i : i +allocation_window] )
                    index =   pd.bdate_range(start =df.index[i] , end = df.index[i] + BDay(allocation_window) ) )
        all_weights = all_weights.append( weights_df)
    all_weights.index.name = 'asofdate'
    all_weights = all_weights.reset_index().drop_duplicates().set_index('asofdate')
    return all_weights


In [143]:
# %%capture
# #search for the best window 

# rst = {}
# for timeframe in [(7,7),(7,360),
#                (90,7),(90,720),
#                (180,7),(180,720)]: 
#     hrp_weights = hrp_weight_generator(timeframe[0],timeframe[1], price)
#     rst[timeframe]=backtest(price.loc['2015-01-01':], hrp_weights.loc['2015-01-01':], 0.02)['portfolio_cum_returns'][-1]

In [144]:
# %%capture
# weights_dic,price_dic,daily_rts_dic = {},{},{}
# for i in [['QQQ','SPY'],['QQQ','SPY','DOW'],['QQQ','SPY','DOW','RUT'],['QQQ','SPY','EEM'],['QQQ','SPY','EEM','DOW']]:
#     price_dic[str(i)], daily_rts_dic[str(i)] = get_price(i)
#     weights_dic[str(i)] = hrp_weight_generator(7,7, daily_rts_dic[str(i)])

In [145]:
# for i in weights_dic.keys(): 
#     print(i)
#     backtest(price_dic[str(i)], weights_dic[str(i)].loc['2006-01-01':], 0.02)['portfolio_cum_returns'][-1]

## A2C

In [146]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import datetime

from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_portfolio import StockPortfolioEnv

from finrl.model.models import DRLAgent
from finrl.trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts

import sys
sys.path.append("../FinRL-Library")

In [147]:


import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gym.Env):
    """A single stock trading environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then 
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step
        

    """
    metadata = {'render.modes': ['human']}

    def __init__(self, 
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,)) 
        # Shape = (34, 30)
        # covariance matrix + technical indicators
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False     
        self.turbulence_threshold = turbulence_threshold        
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        
    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
        # print(actions)

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(),'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()
            
            plt.plot(self.portfolio_return_memory,'r')
            plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))           
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("=================================")
            
            return self.state, self.reward, self.terminal,{}

        else:
            #print("Model actions: ",actions)
            # actions are the portfolio weight
            # normalize to sum of 1
            #if (np.array(actions) - np.array(actions).min()).sum() != 0:
            #  norm_actions = (np.array(actions) - np.array(actions).min()) / (np.array(actions) - np.array(actions).min()).sum()
            #else:
            #  norm_actions = actions
            weights = self.softmax_normalization(actions) 
            #print("Normalized actions: ", weights)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
            #print(self.state)
            # calcualte portfolio return
            # individual stocks' return * weight
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)
            # update portfolio value
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])            
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value 
            #print("Step reward: ", self.reward)
            #self.reward = self.reward*self.reward_scaling

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False 
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]] 
        return self.state
    
    def render(self, mode='human'):
        return self.state
        
    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output

    
    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']
        
        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [148]:
equal_weights =pd.DataFrame(data = 1/len(df_actions.columns), columns = df_actions.columns,
                            index = df_actions.loc['2015-01-01':].index)


In [149]:
backtest(price,equal_weights , 0.02)['portfolio_cum_returns'][-1]

Asset class:
['GLD', 'IEO', 'LQD', 'QQQ', 'SPY']
Mean returns: 0.07245911558068605, total returns: 0.5772091694956394, sharpe: 0.7117632712171701


0.5772091694956394

# Running 

In [150]:
price, daily_rts = get_price(['SPY','QQQ'])

Exception in thread Thread-41:
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py36/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/anaconda/envs/azureml_py36/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/multitasking/__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/yfinance/multi.py", line 168, in _download_one_threaded
    actions, period, interval, prepost, proxy, rounding)
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/yfinance/multi.py", line 183, in _download_one
    rounding=rounding, many=True)
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/yfinance/base.py", line 157, in history
    data = data.json()
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/requests/models.py", line 900,

KeyboardInterrupt: 

In [ ]:
price, daily_rts  = price.resample('W').first(),daily_rts.resample('W').first()

In [ ]:
weights = {}

## hrp

In [ ]:

%%capture
weights['hrp'] = hrp_weight_generator(7,7, daily_rts)
#hrp_weights30 = hrp_weight_generator(30,30, price)

## A2C

In [124]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_turbulence=False,
                    user_defined_feature = False)

df = fe.preprocess_data(df)

# add covariance matrix as states
df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
# look back is one year
lookback=252
for i in range(lookback,len(df.index.unique())):
    data_lookback = df.loc[i-lookback:i,:]
    price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
    return_lookback = price_lookback.pct_change().dropna()
    covs = return_lookback.cov().values 
    cov_list.append(covs)
  
df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)
train = data_split(df, '2009-01-01','2015-01-01')
#trade = data_split(df, '2020-01-01', config.END_DATE)      
    
# # resample weekly
# df['date'] = pd.to_datetime(df.date)
# df = df.set_index('date')


# rst = pd.DataFrame()
# for i in df.tic.unique():
#     subdf = df[df['tic'] == i ]
#     rst = rst.append(subdf.resample('W').first().reset_index())
# df = rst.sort_values(by ='date')    

trade = data_split(df,'2015-01-01', '2021-08-01')


Successfully added technical indicators


In [125]:

%%capture
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "transaction_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)

env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

# initialize

agent = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

trained_a2c = agent.train_model(model=model_a2c, 
                                tb_log_name='a2c',
                                total_timesteps=60000)

In [126]:
trade = data_split(df,'2015-01-01', '2021-08-01')
e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)

df_daily_return, weights['a2c'] = DRLAgent.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym)
weights['a2c'] .index = pd.to_datetime(weights['a2c'] .index)

begin_total_asset:1000000
end_total_asset:2040422.1818950502
Sharpe:  0.8670707288368692
hit end!


## PPO

In [127]:
%%capture
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model(model_name = "ppo",model_kwargs = PPO_PARAMS)

trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=80000)

In [128]:
trade = data_split(df,'2015-01-01', '2021-08-01')
e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)

df_daily_return_ppo, weights['ppo']  = DRLAgent.DRL_prediction(model=trained_ppo,
                        environment = e_trade_gym)
weights['ppo'].index = pd.to_datetime(weights['ppo'].index)

begin_total_asset:1000000
end_total_asset:1864699.69245699
Sharpe:  0.7412741950505064
hit end!


# Performance

In [129]:
weights['hrp']

tic,QQQ,SPY
asofdate,,
1995-02-27,1.000000,0.000000
1995-02-28,1.000000,0.000000
1995-03-01,1.000000,0.000000
1995-03-02,1.000000,0.000000
1995-03-03,1.000000,0.000000
...,...,...
2021-06-16,0.189476,0.810524
2021-06-17,0.189476,0.810524
2021-06-18,0.189476,0.810524


In [130]:
rts = {}
for i in weights.keys(): 
    rts[i] =backtest(price, weights[i].loc['2015-01-01':], 0.02)[['portfolio_cum_returns']]
    rts[i].columns =[ i ]

Asset class:
['QQQ', 'SPY']
Mean returns: 0.07319544967821177, total returns: 0.5700439806151187, sharpe: 0.6079877319136341


Asset class:
['GLD', 'IEO', 'LQD', 'QQQ', 'SPY']
Mean returns: 0.08913544491153212, total returns: 0.7494972212746702, sharpe: 0.8230726115174021


Asset class:
['GLD', 'IEO', 'LQD', 'QQQ', 'SPY']
Mean returns: 0.07478773837545363, total returns: 0.5983495835877344, sharpe: 0.7074260550445618


In [132]:
rtsdf  = pd.concat(rts, axis = 1)

In [133]:
rtsdf.columns= rtsdf.columns.droplevel()

In [134]:
rtsdf.iplot()